### Problem Statement

**Initially:**
To clean and preprocess a mental health-related Twitter dataset for accurate sentiment analysis in future model development.

**Further:**
To train a Machine Learning model on the cleaned dataset and make predictions which will be used to evaluate the model's performance. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Our column names were missing at the start

column_names = ['ID', 'Topic', 'Sentiment', 'Tweet Content']

# assign column names
data = pd.read_csv('twitter.csv', names=column_names, header=None)

print(data.head(10))



     ID        Topic Sentiment  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   
5  2401  Borderlands  Positive   
6  2402  Borderlands  Positive   
7  2402  Borderlands  Positive   
8  2402  Borderlands  Positive   
9  2402  Borderlands  Positive   

                                       Tweet Content  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...  
5  im getting into borderlands and i can murder y...  
6  So I spent a few hours making something for fu...  
7  So I spent a couple of hours doing something f...  
8  So I spent a few hours doing something for fun...  
9  So I spent a few hours making something for fu...  


In [2]:
data.columns
# Data.columns lists the columns in the dataframe, which we set above

Index(['ID', 'Topic', 'Sentiment', 'Tweet Content'], dtype='object')

In [3]:
# Exploratory data analysis
data.dtypes


ID                int64
Topic            object
Sentiment        object
Tweet Content    object
dtype: object

Our dataset consists of objects, which are in this case strings which will need to be processed into something numerical later on.

In [4]:
data.describe()

,ID
count,74682.000000
mean,6432.586165
std,3740.427870
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             74682 non-null  int64 
 1   Topic          74682 non-null  object
 2   Sentiment      74682 non-null  object
 3   Tweet Content  73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


## Data Cleaning ##

This step includes identifying rows containing NAN values, duplicate values, and incorrect sentiment values.

In [6]:
rows_with_nan = data[data.isna().any(axis=1)]  # Rows containing NaN 

duplicate_rows = data[data.duplicated(subset=['Tweet Content'], keep='first')]

valid_sentiments = ['positive', 'neutral', 'negative']
invalid_sentiments = data[~data['Sentiment'].isin(valid_sentiments)]

# Combine all filtered rows 
filtered = pd.concat([rows_with_nan, duplicate_rows, invalid_sentiments]).drop_duplicates()

# removing filtered rows and test
data_cleaned = data.drop(filtered.index)

print(data_cleaned.isin(filtered).any().any())  # Should hopefully return False 



False


In [7]:
import re

def clean_tweet(tweet):
    # Handle non-string inputs and missing values by converting to an empty string
    if not isinstance(tweet, str):
        tweet = ""
    
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)
    
    # Remove mentions (@username)
    tweet = re.sub(r'@\w+', '', tweet)
    
    # Remove hashtags (#hashtag)
    tweet = re.sub(r'#\w+', '', tweet)
    
    # Remove non-alphabetical characters (punctuation, numbers, etc.)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    
    # Convert text to lowercase
    tweet = tweet.lower()
    
    # Remove stopwords
    stop_words = {'a', 'an', 'the', 'in', 'on', 'at', 'for', 'to', 'of', 'is', 'are', 'am', 'was', 'were', 'be'}
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    
    return tweet

# Apply the function to column
data['Cleaned Tweet'] = data['Tweet Content'].apply(clean_tweet)

# Check the cleaned tweets
print(data[['Tweet Content', 'Cleaned Tweet']].head())

                                       Tweet Content  \
0  im getting on borderlands and i will murder yo...   
1  I am coming to the borders and I will kill you...   
2  im getting on borderlands and i will kill you ...   
3  im coming on borderlands and i will murder you...   
4  im getting on borderlands 2 and i will murder ...   

                                       Cleaned Tweet  
0   im getting borderlands and i will murder you all  
1           i coming borders and i will kill you all  
2     im getting borderlands and i will kill you all  
3    im coming borderlands and i will murder you all  
4  im getting borderlands and i will murder you m...  


## Data Visualisation ## 

In [8]:
# import matplotlib.pyplot as plt

# # Scatter Plot
# plt.figure(figsize=(10, 6))
# plt.scatter(data['Cleaned Tweet'], data['Sentiment'], alpha=0.5, color='blue')

# # Add titles and labels
# plt.title('Sentiment Score vs Tweet Content')
# plt.xlabel('Tweet')
# plt.ylabel('Sentiment Score')
# plt.grid(True)
# plt.show()



### Model Selection and Training

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Sample Data
X = data["Cleaned Tweet"]
y = data["Sentiment"]  # Labels

# Feature Extraction
vectoriser = CountVectorizer()
X_vec = vectoriser.fit_transform(X)


In [ ]:
# Train Logistic Regression Model
model = LogisticRegression()
model.fit(X_vec, y)

# Predictions on New Data
new_tweets = [
    "Absolutely fantastic service!", 
    "I hate how bad this was.", 
    "It's neither good nor bad."
]
new_tweets_vec = vectoriser.transform(new_tweets)
predictions = model.predict(new_tweets_vec)

# Output Predictions
for tweet, sentiment in zip(new_tweets, predictions):
    print(f"Tweet: {tweet} -> Sentiment: {sentiment}")

Tweet: Absolutely fantastic service! -> Sentiment: Positive
Tweet: I hate how bad this was. -> Sentiment: Negative
Tweet: It's neither good nor bad. -> Sentiment: Positive


C:\Users\thoma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
